In [1]:

import openslide as osl
import sys
import os
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
import cv2
def create_dir(path):
    import os
    if not os.path.exists(path):
        os.makedirs(path)

In [23]:
mpp=0.5
root_dir='../../data/pdl1_wsi_anno/'
save_wsi_path='../../data/IHC4BC_Compressed/external_patches/'
image_size=1000
image_resize_size=512

json_list=glob('../../data/pdl1_wsi_anno/PD-L1_tumor_area/*.json')
for i in tqdm(range(38,len(json_list))):
    json_path=json_list[i]
    with open(json_path) as f:
        data=json.load(f)
    slide_mpp=float(data['mpp_X'])
    try:
        wsi_name=data['filename'].replace('.tiff',data['origin_extension']).replace( 'WSI-STCA_PDL1','CODIPAI-STBX')
        wsi_path=os.path.join(root_dir,'PD-L1(22C3)',wsi_name)
        slide_image=osl.open_slide(wsi_path)
    except:
        wsi_name=data['filename'].replace('.tiff',data['origin_extension']).replace( 'WSI-STCA_PDL1','CODIPAI-STOP')
        wsi_path=os.path.join(root_dir,'PD-L1(22C3)',wsi_name)
        slide_image=osl.open_slide(wsi_path)
    wsi_w, wsi_h=slide_image.dimensions
    thumbnail=slide_image.get_thumbnail((wsi_w//20, wsi_h//20))
    annotation_mask=np.zeros_like(np.array(thumbnail)[:,:,0], dtype=np.uint8)
    
    for region in data['objects']:
        points=region['coordinate']
        scaled_points=[(int(x//20), int(y//20)) for x,y in points]
        cv2.fillPoly(annotation_mask, [np.array(scaled_points)], 255)
        
    blur = cv2.GaussianBlur(np.array(thumbnail), (5,5), 0)
    thumbnail_mask=np.zeros_like(annotation_mask)
    tre,thumbnail_mask=cv2.threshold(blur[:,:,0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thumbnail_mask=255-thumbnail_mask
    common_mask=cv2.bitwise_and(thumbnail_mask, thumbnail_mask, mask=annotation_mask.astype(np.uint8))
    common_mask=(common_mask > 0).astype(np.uint8)
    slide_patch_size=int(image_size * (mpp/slide_mpp))
    num_row=wsi_h//slide_patch_size
    num_col=wsi_w//slide_patch_size
    mask_patch_size=slide_patch_size//20
    for col in range(num_col):
        for row in range(num_row):
            x1=col*slide_patch_size
            y1=row*slide_patch_size
            if common_mask[row*mask_patch_size:(row+1)*mask_patch_size, col*mask_patch_size:(col+1)*mask_patch_size].mean()>0.4:
                patch=slide_image.read_region((col*slide_patch_size, row*slide_patch_size), 0, (slide_patch_size, slide_patch_size))
                patch=patch.convert('RGB')
                patch=patch.resize((image_resize_size, image_resize_size), Image.LANCZOS)
                create_dir(save_wsi_path)    
                patch.save(os.path.join(save_wsi_path, f"{os.path.splitext(wsi_name)[0]}_patch{slide_patch_size}_{col*slide_patch_size}_{row*slide_patch_size}.tiff"))
                

100%|██████████| 40/40 [5:19:56<00:00, 479.91s/it]   


In [22]:
json_path

'../../data/pdl1_wsi_anno/PD-L1_tumor_area/cancel_list.json'